In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load data
df = pd.read_csv('../archive/survey.csv')

# Drop columns not useful for prediction
df = df.drop(['Timestamp', 'comments', 'state'], axis=1)

# Standardize gender values
df['Gender'] = df['Gender'].str.lower().replace({
    'male': 'male', 'm': 'male', 'male-ish': 'male', 'maile': 'male',
    'female': 'female', 'f': 'female', 'cis female': 'female', 'trans-female': 'female'
})
df['Gender'] = df['Gender'].where(df['Gender'].isin(['male', 'female']), 'other')

# Fill missing values
for col in df.columns:
    if df[col].dtype == 'O':
        fill_value = str(df[col].mode()[0])
        df[col] = df[col].fillna(fill_value)
    else:
        fill_value = float(df[col].median())
        df[col] = df[col].fillna(fill_value)

# Encode categorical variables
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Define target and features
target = 'treatment'  # or 'diagnosis' if available
X = df.drop(target, axis=1)
y = df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure correct types for downstream use
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.Series(y_train)
y_test = pd.Series(y_test)

# Save splits
X_train.to_csv('../archive/X_train.csv', index=False)
X_test.to_csv('../archive/X_test.csv', index=False)
y_train.to_csv('../archive/y_train.csv', index=False)
y_test.to_csv('../archive/y_test.csv', index=False)

print(X_train.shape, X_test.shape)
print(y_train.value_counts(), y_test.value_counts())

(1007, 23) (252, 23)
treatment
1    514
0    493
Name: count, dtype: int64 treatment
0    129
1    123
Name: count, dtype: int64
